In [1]:
import pandas as pd

loans = pd.read_csv("filtered_loans_2007.csv")

In this project, we'll prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process.

This is because the mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values. To reinforce this requirement, **scikit-learn** will return an error if we try to train a model using data that contain missing values or non-numeric values when working with models like **linear regression and logistic regression**.

In [2]:
null_counts = loans.isnull().sum()
null_counts

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

While most of the columns have no missing values, two columns have fifty or less rows with missing values, and two columns, `emp_length` and `pub_rec_bankruptcies`, contain a relatively high amount of missing values.

Domain knowledge tells us that employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values.

Let's inspect the values of the column `pub_rec_bankruptcies`

In [3]:
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


We see that this column offers very little variability, nearly `94%` of values are in the same category. It probably won't have much predictive value. let's drop it. In addition, we'll remove the remaining rows containing null values.

In [4]:
loans = loans.drop("pub_rec_bankruptcies", axis = 1)

loans = loans.dropna(axis = 0)

In [5]:
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


While the numerical columns can be used natively with **scikit-learn**, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth. 

In [6]:
object_columns_df =loans.select_dtypes(include = ["object"])
object_columns_df.head(2)

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013


Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

* `home_ownership`: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
* `verification_status`: indicates if income was verified by Lending Club,
* `emp_length`: number of years the borrower was employed upon time of application,
* `term`: number of payments on the loan, either 36 or 60,
* `addr_state`: borrower's state of residence,
* `purpose`: a category provided by the borrower for the loan request,
* `title`: loan title provided the borrower,

There are also some columns that represent numeric values, that need to be converted:

* `int_rate`: interest rate of the loan in %,
* `revol_util`: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit.

Based on the first row's values for **purpose** and **title**, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.

In [7]:
print(loans["title"].value_counts())
print(loans["purpose"].value_counts())

Debt Consolidation                       2068
Debt Consolidation Loan                  1599
Personal Loan                             624
Consolidation                             488
debt consolidation                        466
Credit Card Consolidation                 345
Home Improvement                          336
Debt consolidation                        314
Small Business Loan                       298
Credit Card Loan                          294
Personal                                  290
Consolidation Loan                        250
Home Improvement Loan                     228
personal loan                             219
Loan                                      202
Wedding Loan                              199
personal                                  198
Car Loan                                  188
consolidation                             186
Other Loan                                168
Wedding                                   148
Credit Card Payoff                

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

* `earliest_cr_line`: The month the borrower's earliest reported credit line was opened,
* `last_credit_pull_d`: The most recent month Lending Club pulled credit for this loan.

Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

In [8]:
loans = loans.drop(["last_credit_pull_d", "addr_state", "title", "earliest_cr_line"], axis = 1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")

In [9]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term']

for col in cols:
    print(loans[col].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64


The `home_ownership`, `verification_status`, `emp_length`, `term`, and `addr_state` columns all contain multiple discrete values. We should clean the `emp_length` column and treat it as a numerical one since the values have ordering (**2 years of employment is less than 8 years**).

In [10]:
loans["emp_length"].value_counts(dropna = False)

10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64

In [11]:
mapping_dict = {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }

# mapping_dict = {
#     "emp_length": {
#         "10+ years": 10,
#         "9 years": 9,
#         "8 years": 8,
#         "7 years": 7,
#         "6 years": 6,
#         "5 years": 5,
#         "4 years": 4,
#         "3 years": 3,
#         "2 years": 2,
#         "1 year": 1,
#         "< 1 year": 0,
#         "n/a": 0
#     }
# }

In [12]:
loans["emp_length"] = loans["emp_length"].map(mapping_dict)
# loans = loans.replace(mapping_dict) -----> alternative method using second dictionary

In [13]:
loans["emp_length"].value_counts(dropna =False)

10    8545
0     4513
2     4303
3     4022
4     3353
5     3202
1     3176
6     2177
7     1714
8     1442
9     1228
Name: emp_length, dtype: int64

Let's now encode the `home_ownership`, `verification_status`, `purpose`, and `term` columns as dummy variables so we can use them in our model.

In [14]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]

dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis = 1)
loans.drop(cat_columns, axis = 1, inplace = True)

In this project, we performed the last amount of data preparation necessary to start training machine learning models. We converted all of the columns to numerical values because those are the only type of value **scikit-learn** can work with. In the next file, we'll experiment with training models and evaluating accuracy using cross-validation.

In [15]:
loans.to_csv("cleaned_loans_2007.csv", sep = ",", index = False)